In [1]:
# Importing packages
import numpy as np
from sympy import solve, symbols, Eq
import random 
from tqdm import tqdm

# Importing openfermion
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev

# Importing Pauli matrices
from pauli_matrices import I, X, Y, Z

C:\Users\Bowy-\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Bowy-\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Bowy-\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [7]:
def get_Pauli_chain(list_w_matrices):

    # Initializing the Pauli chain
    Pauli_chain = list_w_matrices[0]

    # Adding the additional terms
    for matrix in list_w_matrices[1:]:
        Pauli_chain = np.kron(Pauli_chain, matrix)

    # returning the final Pauli chain
    return Pauli_chain




def get_Bell_term(measurement_choices, measurement_indices):
            
    # Obtaining Pauli chain
    P_chain = get_Pauli_chain(measurement_choices)

    # Obtaining the variable indices
    lower_string = ''

    for j in measurement_indices:
        lower_string += str(j)

    # Finilizing the variable
    var = symbols('c_{' + lower_string + '}')

    # Returning the Bell operator term
    return var * P_chain, var



In [8]:
# Defining number of molecular orbitals/qubits
N = 4

# Let us define a molecular Hamiltonian that we can use to feed it to openfermion
h11 = h22 = -1.252477
h33 = h44 = -0.475934
v1221 = v2112 = 0.674493
v3443 = v4334 = 0.697397
v1331 = v2442 = v2332 = v2442 = v3113 = v1441 = v3223 = v4224 = 0.663472
v1313 = v2424 = v3241 = v3421 = v1423 = v1234 = 0.181287

# Initializing single particle and Coulomb operators
h_single_and_coulomb = np.array([
    [h11, v1221, v1331-v1313, v1441       ],
    [0,   h22,   v2332,       v2442-v2424 ],
    [0,   0,     h33,         v3443       ],
    [0,   0,     0,           h44         ]
]) 

# Initializing the Hamiltonian
hamiltonian_fermionic_op = 0

# Adding the Coulom interactions to the Hamiltonian
for i in range(N):
    # Adding single Hamiltonian terms
    hamiltonian_fermionic_op += FermionOperator(str(i)+'^ '+str(i), h_single_and_coulomb[i, i])

    # Adding coulomb terms
    for j in range(i+1, N):
        hamiltonian_fermionic_op += FermionOperator(str(j) + '^ ' + str(j) + ' ' + str(i) + '^ ' + str(i), h_single_and_coulomb[i, j])

# Adding the double excitation operators
hamiltonian_fermionic_op += (FermionOperator('0^ 1^ 3 2', v1234) + FermionOperator('2^ 3^ 1 0', v1234) +
            FermionOperator('0^ 3^ 1 2', v1234) + FermionOperator('2^ 1^ 3 0', v1234)
)

# Calculating the Jordan Wigner Hamiltonian
qubit_hamiltonian = jordan_wigner(hamiltonian_fermionic_op)

# Initializing the pauli matrices and strings
pauli_matrices = [I, X, Y, Z]
pauli_strings = ['I', 'X', 'Y', 'Z']

# Initializing the hamiltonian
H = np.zeros((2**N, 2**N))

# Calculating the Hamiltonian
for key in qubit_hamiltonian.terms.keys():
    # Initializing the indices
    idx = [0, 0, 0, 0]

    # Updating the possible indices
    for i in range(len(key)):
        for j in range(len(pauli_strings)):
            if key[i][1] == pauli_strings[j]:
                idx[key[i][0]] = j

    # Defining the Pauli chain
    matrices = [pauli_matrices[k] for k in idx]
    p_chain = get_Pauli_chain(matrices)

    # Adding terms to the Hamiltonian
    H += np.real( qubit_hamiltonian.terms[key] * np.array(p_chain) )


# Calculating the eigenvalues and eigenstates
eig_vals, eig_vecs = np.linalg.eigh(H)

# Extracting the ground state energy and the respective eigenstate
E_G = eig_vals[0]
psi_G = eig_vecs[:,0]
psi_G_dagger = np.conjugate(np.transpose(psi_G))



In [9]:
# Make a matrix M such that the parties can choose from the measurements in which they measure
# This makes it a matrix with dimensions nxm+1, where the extra term accounts for the identity matrix
# Let's say we construct the M matrix as 

# Defining measurements of the parties
M1 = M2 = M3 = M4 = [I, X, Y, Z]

# Consrtucting a measurement matrix
M = np.array([
    M1,
    M2,
    M3,
    M4
])

# Initializing the Bell operator and a string containing the variables
B = 0
var = []

for i in tqdm(range(N_parties)):

    for j in range(N_parties):

        for k in range(N_parties):

            for l in range(N_parties):

                # Constructing the measurement choices
                measurement_indices = [i, j, k, l]
                measurement_choices = [M[p, measurement_indices[p]] for p in range(N_parties)]

                # Calculating the Bell terms and the corresponding variable
                B_temp, var_temp = get_Bell_term(measurement_choices, measurement_indices)
                
                # Adding these temporaries to the existsing Bell operator and the var list
                B += B_temp
                var.append(var_temp)
# 

-1.8510456784448643